In [8]:
from decision_rules.measures import c2

from rules_to_cnf.generate_cnf_ruleset import (
    prepare_dataset,
    create_rulekit_ruleset,
    convert_rulekit_to_decision_rules,
    load_ruleset,
)
from rules_to_cnf.utils.ruleset_quality_report import create_full_report
from rules_to_cnf.converters.sympy_cnf_converter import SympyCNFConverter
from rules_to_cnf.converters.nnf_cnf_converter import NnfCNFConverter

# Load dataset

In [9]:
X, y = prepare_dataset("iris.csv")

# Generate ruleset with RuleKit

In [10]:
rulekit_model = create_rulekit_ruleset(X, y)

In [11]:
for rule in rulekit_model.model.rules:
    print(rule)

IF petallength = (-inf, 2.45) THEN class = {Iris-setosa}
IF sepalwidth = (-inf, 2.95) AND petallength = (-inf, 4.75) AND sepallength = <4.95, inf) THEN class = {Iris-versicolor}
IF petallength = <2.45, 4.75) AND petalwidth = (-inf, 1.65) THEN class = {Iris-versicolor}
IF sepalwidth = (-inf, 2.95) AND petalwidth = (-inf, 1.65) AND petallength = (-inf, 4.95) AND sepallength = <4.70, inf) THEN class = {Iris-versicolor}
IF petallength = <2.45, 5.35) AND petalwidth = (-inf, 1.65) THEN class = {Iris-versicolor}
IF petallength = <2.45, 5.05) AND petalwidth = (-inf, 1.85) AND sepallength = <4.95, inf) THEN class = {Iris-versicolor}
IF petalwidth = <1.75, inf) AND petallength = <4.85, inf) THEN class = {Iris-virginica}
IF petalwidth = <1.75, inf) THEN class = {Iris-virginica}
IF petallength = <4.95, inf) THEN class = {Iris-virginica}
IF petalwidth = <1.65, inf) THEN class = {Iris-virginica}


In [12]:
decision_rules_ruleset = convert_rulekit_to_decision_rules(rulekit_model, X, y)

In [13]:
for rule in decision_rules_ruleset.rules:
    print(rule)

IF petallength < 2.45 THEN class = Iris-setosa (p=50, n=0, P=50, N=100)
IF sepalwidth < 2.95 AND petallength < 4.75 AND sepallength >= 4.95 THEN class = Iris-versicolor (p=30, n=0, P=50, N=100)
IF petallength >= 2.45 AND petallength < 4.75 AND petalwidth < 1.65 THEN class = Iris-versicolor (p=44, n=0, P=50, N=100)
IF sepalwidth < 2.95 AND petalwidth < 1.65 AND petallength < 4.95 AND sepallength >= 4.70 THEN class = Iris-versicolor (p=33, n=0, P=50, N=100)
IF petallength >= 2.45 AND petalwidth < 1.65 AND petallength < 5.35 THEN class = Iris-versicolor (p=48, n=2, P=50, N=100)
IF petallength >= 2.45 AND petallength < 5.05 AND petalwidth < 1.85 AND sepallength >= 4.95 THEN class = Iris-versicolor (p=48, n=5, P=50, N=100)
IF petalwidth >= 1.75 AND petallength >= 4.85 THEN class = Iris-virginica (p=43, n=0, P=50, N=100)
IF petalwidth >= 1.75 THEN class = Iris-virginica (p=45, n=1, P=50, N=100)
IF petallength >= 4.95 THEN class = Iris-virginica (p=44, n=2, P=50, N=100)
IF petalwidth >= 1.65 

# Convert ruleset to CNF

## Using SymPy

In [14]:
converter = SympyCNFConverter()

In [15]:
sympy_cnf = converter.convert_to_cnf(decision_rules_ruleset)

Converting rules for conclusion class = Iris-setosa to sympy...
Converting DNF to CNF...
Converting CNF to decision rules...
Conversion finished.

Converting rules for conclusion class = Iris-versicolor to sympy...
Converting DNF to CNF...
Converting CNF to decision rules...
Conversion finished.

Converting rules for conclusion class = Iris-virginica to sympy...
Converting DNF to CNF...
Converting CNF to decision rules...
Conversion finished.

Creating ruleset out of 3 rules...


## Using NNF

In [16]:
converter = NnfCNFConverter()

In [17]:
nnf_cnf = converter.convert_to_cnf(decision_rules_ruleset)

Converting rules for conclusion class = Iris-setosa to nnf objects...
Converting DNF to CNF...
Converting CNF to decision rules...
Conversion finished.

Converting rules for conclusion class = Iris-versicolor to nnf objects...
Converting DNF to CNF...
Converting CNF to decision rules...
Conversion finished.

Converting rules for conclusion class = Iris-virginica to nnf objects...
Converting DNF to CNF...
Converting CNF to decision rules...
Conversion finished.

Creating ruleset out of 3 rules...


# Comparing the original ruleset and the CNF ruleset

After conversion you can generate a text report to compare between the original ruleset and the CNF ruleset.

In [18]:
# perform some necessary steps after creating the CNF ruleset
sympy_cnf.update(X, y, c2)
sympy_cnf.set_prediction_strategy("vote")

# define metrics to be calculated
METRICS = ["precision", "coverage", "C2"]

create_full_report(
    dnf_ruleset=decision_rules_ruleset,
    cnf_ruleset=sympy_cnf,
    X=X,
    y=y,
    path="./iris_report.txt",
    preambule=""
)